# 아영 - Story Generator

# 1. Ready

## 1.1 기본 패키지 임포트

In [1]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel


## 1.2 학습 모델 다운로드

In [2]:
MODEL_NAME = "skt/kogpt2-base-v2"

### 토크나이저 load

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)

TOKENS_DICT = {
    'bos_token':'<s>',
    'eos_token':'</s>',
    'unk_token':'<unk>',
    'pad_token':'<pad>',
    'mask_token':'<mask>'
}

# 특수 토큰이 토크나이저에 추가되고 모델은 수정된 토크나이저에 맞게 임베딩의 크기를 조정
tokenizer.add_special_tokens(TOKENS_DICT)

print(tokenizer.special_tokens_map)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


### 미세조정학습 모델 load

In [4]:
from util.generator import sample_sequence as gen

In [5]:
from util.model import model_loading as load

In [6]:
checkpointPath = "./modelCheckpoint/talestart680w.tar"
loading = True



"""
talestart_from1980_newdata 은호
KoGPT2_checkpoint_novelstart 지현
tale-novel-batch-3-1 창균
talestart680w 현지
travel2 아영
"""

'\ntalestart_from1980_newdata 은호\nKoGPT2_checkpoint_novelstart 지현\ntale-novel-batch-3-1 창균\ntalestart680w 현지\ntravel2 아영\n'

In [7]:
model,ckp = load(checkpointPath, PU = 'cpu', status = loading)

In [8]:
print(ckp['epoch'], ckp['loss'])

870 tensor(0.0356, requires_grad=True)


In [9]:
vocab_path = "./data/정제/index_tale_plus_novel.txt"

In [10]:
p1, p2, p3 = 1,2,1

max_sent = [p1,p2//2, p2//2 ,p3]

context = "어제"


In [11]:
from util.generator import sample_sequence
from util.postprocessing import paragraph_similarity

In [13]:
result = []
pre_p = sample_sequence(vocab=vocab_path,
                    model=model, 
                     length = max_sent[0]*30,
                     context=context,
                     num_samples=1, 
                     repetition_penalty=2.0,
                     top_p=0.9,
                     tokenizer = tokenizer)
pre_p = tokenizer.decode(pre_p[0,:].tolist())
result.append(prep_p)

31it [00:02, 12.20it/s]                                                                                                

Word not in corpus => 0 skipped


In [14]:

import numpy as np
for paragraph in range(len(max_sent[1:])):
    print(f"{paragraph+1}번째 문단")
    for ind in range(max_sent[paragraph]):
        fpass = False
        similarity_simple = 0
        next_p = sample_sequence(vocab= vocab_path,
                        model=model, 
                         length = max_sent[paragraph+1]*30,
                         context=context,
                         num_samples=1, 
                         repetition_penalty=2.0,
                         top_p=0.9,
                         tokenizer = tokenizer)
        next_p = tokenizer.decode(next_p[0,:].tolist())
        next_p = next_p.split('.')
        next_p = next_p[:max_sent[paragraph+1]]
        next_p = ['.'+x for x in next_p]
        next_p = " ".join(next_p)
        similarity = paragraph_similarity(pre_p, next_p)
        while(fpass == False):
            next_p = sample_sequence(vocab= vocab_path,
                        model=model, 
                         length = max_sent[paragraph+1]*30,
                         context=context,
                         num_samples=1, 
                         repetition_penalty=2.0,
                         top_p=0.9,
                         tokenizer = tokenizer)
            next_p = tokenizer.decode(next_p[0,:].tolist())
            similarity = paragraph_similarity(pre_p, next_p)
            fpass = similarity<=0.02
            print(fpass)
        result.append(next_p)
        context = "내일"
        pre_p = next_p

1번째 문단


31it [00:02, 11.86it/s]                                                                                                


Word not in corpus => 0 skipped


31it [00:02, 12.35it/s]                                                                                                


Word not in corpus => 0 skipped
False


...

Word not in corpus => 0 skipped
False


31it [00:02, 11.29it/s]                                                                                                


Word not in corpus => 0 skipped
False


 10%|████████▎                                                                          | 3/30 [00:00<00:01, 14.75it/s]


KeyboardInterrupt: 

In [15]:
print(result)

[]
